In [1]:
# IMPORT PACKAGES

## tweepy - api to get data from twitter
import tweepy as tw
## hiding API keys - ref: https://youtu.be/CJjSOzb0IYs
import apikeys
## data manipulation
import pandas as pd
## date transformation
from datetime import datetime, timedelta

In [2]:
# CREATE CONNECTION WITH TWITTER API

## Authentication Tokens
my_bearer = apikeys.bearer
my_key = apikeys.key
my_secret = apikeys.secret
my_token = apikeys.token
my_token_secret = apikeys.token_secret

## creating client object
client = tw.Client(
    bearer_token=my_bearer, 
    consumer_key=my_key, 
    consumer_secret=my_secret, 
    access_token=my_token, 
    access_token_secret=my_token_secret
    )

## creating API object
# authorization of consumer key and consumer secret
auth = tw.OAuthHandler(
    consumer_key=my_key, 
    consumer_secret=my_secret
    )
# set access to user's access key and access secret 
auth.set_access_token(my_token, my_token_secret)
# calling the api 
api = tw.API(auth)

In [3]:
# PROFILE QUANTITY MENTIONS

# Replace with your own search query
query_list = ['@LulaOficial', '@jairbolsonaro', '@cirogomes', '@simonetebetbr']
query_filter = ' -is:retweet'

## all recent tweets mentions
counts_recent = client.get_recent_tweets_count(query=query_list[0], granularity='hour')
## recent tweets mentions without retweets
counts_recent_filtered = client.get_recent_tweets_count(query=query_list[0] + query_filter, granularity='hour')

## creating the dataframe and removing the first and the last range
df_count = pd.DataFrame(counts_recent[0], columns=['start', 'end', 'tweet_count']).rename(columns={"tweet_count": query_list[0] + '_mentions'}).iloc[1:len(counts_recent[0])-1].reset_index(drop=True)
## convert date columns from string to datetime
df_count['start'] = pd.to_datetime(df_count['start'], utc=True).map(lambda x: x.tz_convert('America/Bahia'))
df_count['end'] = pd.to_datetime(df_count['end'], utc=True).map(lambda x: x.tz_convert('America/Bahia'))
## split datetime into date and time
df_count['start_date'] = pd.to_datetime(df_count['start']).dt.date
df_count['start_time'] = pd.to_datetime(df_count['start']).dt.time
df_count['end_date'] = pd.to_datetime(df_count['end']).dt.date
df_count['end_time'] = pd.to_datetime(df_count['end']).dt.time
## reorder the columns sequence
df_count = df_count.loc[:, ['start', 'start_date', 'start_time', 'end', 'end_date', 'end_time', query_list[0] + '_mentions']]
## get tweet count without retweets
df_temp = pd.DataFrame(counts_recent_filtered[0], columns=['tweet_count']).rename(columns={"tweet_count": query_list[0] + '_mentions_without_retweet'}).iloc[1:len(counts_recent[0])-1].reset_index(drop=True)
df_count = pd.concat([df_count, df_temp], axis=1)
## check numeric columns type - fillna and convert to integer
df_count[query_list[0] + '_mentions'] = df_count[query_list[0] + '_mentions'].fillna(0).astype(int)
df_count[query_list[0] + '_mentions_without_retweet'] = df_count[query_list[0] + '_mentions_without_retweet'].fillna(0).astype(int)

## loop - get data about the top 4 candidates
for i in range(1, len(query_list)):
    
    ## all recent tweets mentions
    counts_recent = client.get_recent_tweets_count(query=query_list[i], granularity='hour')
    ## recent tweets mentions without retweets
    counts_recent_filtered = client.get_recent_tweets_count(query=query_list[i] + query_filter, granularity='hour')

    ## get tweet count with retweets
    df_temp = pd.DataFrame(counts_recent[0], columns=['tweet_count']).rename(columns={"tweet_count": query_list[i] + '_mentions'}).iloc[1:len(counts_recent[0])-1].reset_index(drop=True)
    df_count = pd.concat([df_count, df_temp], axis=1)
    
    ## get tweet count without retweets
    df_temp = pd.DataFrame(counts_recent_filtered[0], columns=['tweet_count']).rename(columns={"tweet_count": query_list[i] + '_mentions_without_retweet'}).iloc[1:len(counts_recent[0])-1].reset_index(drop=True)
    df_count = pd.concat([df_count, df_temp], axis=1)
    
    ## check numeric columns type - fillna and convert to integer
    df_count[query_list[i] + '_mentions'] = df_count[query_list[i] + '_mentions'].fillna(0).astype(int)
    df_count[query_list[i] + '_mentions_without_retweet'] = df_count[query_list[i] + '_mentions_without_retweet'].fillna(0).astype(int)

## final dataset
df_count

,start,start_date,start_time,end,end_date,end_time,@LulaOficial_mentions,@LulaOficial_mentions_without_retweet,@jairbolsonaro_mentions,@jairbolsonaro_mentions_without_retweet,@cirogomes_mentions,@cirogomes_mentions_without_retweet,@simonetebetbr_mentions,@simonetebetbr_mentions_without_retweet
0,2022-09-12 19:00:00-03:00,2022-09-12,19:00:00,2022-09-12 20:00:00-03:00,2022-09-12,20:00:00,12009,6035,7892,2065,3210,1538,253,180
1,2022-09-12 20:00:00-03:00,2022-09-12,20:00:00,2022-09-12 21:00:00-03:00,2022-09-12,21:00:00,22107,8869,6958,2129,2604,1082,237,164
2,2022-09-12 21:00:00-03:00,2022-09-12,21:00:00,2022-09-12 22:00:00-03:00,2022-09-12,22:00:00,23491,10706,7819,2431,3043,1685,358,227
3,2022-09-12 22:00:00-03:00,2022-09-12,22:00:00,2022-09-12 23:00:00-03:00,2022-09-12,23:00:00,25729,12866,7137,2191,2120,1258,329,241
4,2022-09-12 23:00:00-03:00,2022-09-12,23:00:00,2022-09-13 00:00:00-03:00,2022-09-13,00:00:00,17254,8781,5527,1549,1604,885,241,157
5,2022-09-13 00:00:00-03:00,2022-09-13,00:00:00,2022-09-13 01:00:00-03:00,2022-09-13,01:00:00,9560,4682,7533,1972,991,565,190,141
6,2022-09-13 01:00:00-03:00,2022-09-13,01:00:00,2022-09-13 02:00:00-03:00,2022-09-13,02:00:00,4427,2287,3248,861,559,285,106,82
7,2022-09-13 02:00:00-03:00,2022-09-13,02:00:00,2022-09-13 03:00:00-03:00,2022-09-13,03:00:00,2512,1204,1685,360,324,154,37,33
8,2022-09-13 03:00:00-03:00,2022-09-13,03:00:00,2022-09-13 04:00:00-03:00,2022-09-13,04:00:00,1590,748,1066,251,207,108,40,24
9,2022-09-13 04:00:00-03:00,2022-09-13,04:00:00,2022-09-13 05:00:00-03:00,2022-09-13,05:00:00,1524,766,1107,248,192,96,31,18


In [4]:
# PROFILE INFORMATION

# Replace with your own users id 
## @LulaOficial id = 2670726740
## @jairbolsonaro id = 128372940 
## @cirogomes id = 33374761
## @simonetebetbr id = 2508415207
user_id_list = [2670726740, 128372940, 33374761, 2508415207]

# fetching the user
user = api.get_user(user_id=user_id_list[0])
# creating dataframe
df_users = pd.DataFrame([datetime.today().strftime('%Y-%m-%d')], columns=['date'])
# fetching the statuses_attributes
df_users['screen_name'] = user.screen_name
df_users['followers'] = user.followers_count
df_users['following'] = user.friends_count
df_users['posts'] = user.statuses_count
df_users['lists'] = user.listed_count
df_users['likes'] = user.favourites_count

## loop - get data about the top 4 candidates
for i in range(1, len(user_id_list)):
    # fetching the user
    user = api.get_user(user_id=user_id_list[i])
    # fetching the statuses_attributes
    df_temp =  pd.DataFrame([datetime.today().strftime('%Y-%m-%d')], columns=['date'])
    # fetching the statuses_attributes
    df_temp['screen_name'] = user.screen_name
    df_temp['followers'] = user.followers_count
    df_temp['following'] = user.friends_count
    df_temp['posts'] = user.statuses_count
    df_temp['lists'] = user.listed_count
    df_temp['likes'] = user.favourites_count

    # adding the info about the new user
    df_users = pd.concat([df_users, df_temp])

df_users = df_users.reset_index(drop=True)
df_users

,date,screen_name,followers,following,posts,lists,likes
0,2022-09-19,LulaOficial,4373145,2805,27058,4066,14991
1,2022-09-19,jairbolsonaro,8933479,497,14978,8415,5886
2,2022-09-19,cirogomes,1488679,415,12764,3278,5837
3,2022-09-19,simonetebetbr,366207,239,3069,853,12423


In [5]:
# Saving dataset
## mentions dataset
df_count.to_csv('datasets/profile_mentions_'+ str(df_count.loc[len(df_count)-1, 'end_date']) +'.csv', index=False)
## user dataset
df_users.to_csv('datasets/profile_info_'+ str(df_users.loc[0, 'date']) +'.csv', index=False)
## last valid date and time
df_count.loc[len(df_count)-1:, ['end_date', 'end_time']].reset_index(drop=True).to_csv('datasets/last_valid_date.csv', index=False)